<a href="https://colab.research.google.com/github/ArmaanSethi/Gemini-Soccer-Scout/blob/main/AI_Soccer_Scout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd

from datetime import datetime
from bs4 import BeautifulSoup
from google.colab import userdata



In [ ]:
# Lamine Yamal
player_name = 'Lamine Yamal'
url = 'https://fbref.com/en/players/82ec26c1/Lamine-Yamal'
attrs = 'scout_summary_AM'


In [ ]:
df = pd.read_html(
    url,
    attrs={'id': attrs} # Change the id & class to correspond to the table
)[0]

In [ ]:
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')


In [ ]:
# Get the player's age, team, league, and position
position = soup.select_one('p:-soup-contains("Position")').text.split(':')[-2].split(' ')[0].strip()
birthday = soup.select_one('span[id="necro-birth"]').text.strip()

# get the player's age
age = (datetime.now() - datetime.strptime(birthday, '%B %d, %Y')).days // 365
team = soup.select_one('p:-soup-contains("Club")').text.split(':')[-1].strip()


In [ ]:
df = df.dropna(subset='Statistic')
df

,Statistic,Per 90,Percentile
0,Non-Penalty Goals,0.15,24.0
1,npxG: Non-Penalty xG,0.24,58.0
2,Shots Total,2.61,71.0
3,Assists,0.22,63.0
4,xAG: Exp. Assisted Goals,0.28,83.0
5,npxG + xAG,0.51,74.0
6,Shot-Creating Actions,4.49,71.0
8,Passes Attempted,44.58,71.0
9,Pass Completion %,78.6%,72.0
10,Progressive Passes,3.38,36.0


In [93]:
simple_prompt = f"""

I need you to create a scouting report on {player_name}. Can you provide me with a summary of their strengths and weaknesses?

Here is the data I have on him:

Player: {player_name}
Position: {position}
Age: {age}
Team: {team}

{df.to_markdown()}


Return the scouting report in the following markdown format:

# Scouting Report for {player_name}

## Strengths
< a list of 1 to 3 strengths >

## Weaknesses
< a list of 1 to 3 weaknesses >

## Summary
< a brief summary of the player's overall performance and if he would be beneficial to the team >
"""

scouting_information = f"""
**Scouting Report Request**

**Player Information:**

* Name: {player_name}
* Position: {position}
* Age: {age}
* Team: {team}
* Statistical Data:
{df.to_markdown()}
"""

video_links = ""
optional_additional_context = f"""
**Additional Context (Optional):**

* Recent Match Footage Link(s): {video_links}
* Specific Areas of Focus: (e.g., tactical awareness, finishing, defensive positioning)
* Target Playing Style/Formation: (to assess fit)
* Team Needs: (to gauge value to your specific squad)
"""

scouting_requirements = f"""
**Scouting Report Requirements:**

1. **Strengths and Weaknesses:**
* **Technical:** Dribbling, passing, shooting, first touch, etc.
* **Tactical:** Positioning, decision-making, reading the game, etc.
* **Physical:** Speed, agility, strength, stamina, etc.
* **Mental:** Work rate, temperament, leadership potential, coachability, etc.

2. **Potential:**
* Estimate the player's potential for development.
* Identify areas where the player could improve.

3. **Injury History:**
* Highlight any significant injuries or recurring issues.

4. **Statistical Analysis:**
* Key performance indicators (KPIs) relevant to the player's position (goals, assists, tackles, pass completion, etc.).
* Contextualize the player's statistics within their league and team.

5. **Comparison:**
* If applicable, compare the player to similar players in their position or league.

6. **Summary:**
* Provide a concise evaluation of the player's overall ability and potential value to a team.

7. **Recommendation:**
 * Offer a clear recommendation on whether the player is worth pursuing further (e.g., scouting, signing, loaning).

**Report Format:**

Please deliver the scouting report in a well-structured Markdown format, using headings, bullet points, and tables for clarity and readability.
Incorporate any relevant video footage analysis for a comprehensive assessment.
"""

complex_prompt = scouting_information + scouting_requirements

In [94]:
use_simple_prompt = False

In [95]:
prompt = simple_prompt if use_simple_prompt else complex_prompt
print(prompt)


**Scouting Report Request**

**Player Information:**

* Name: Lamine Yamal
* Position: 
* Age: 16
* Team: Barcelona
* Statistical Data:
|    | Statistic                | Per 90   |   Percentile |
|---:|:-------------------------|:---------|-------------:|
|  0 | Non-Penalty Goals        | 0.15     |           24 |
|  1 | npxG: Non-Penalty xG     | 0.24     |           58 |
|  2 | Shots Total              | 2.61     |           71 |
|  3 | Assists                  | 0.22     |           63 |
|  4 | xAG: Exp. Assisted Goals | 0.28     |           83 |
|  5 | npxG + xAG               | 0.51     |           74 |
|  6 | Shot-Creating Actions    | 4.49     |           71 |
|  8 | Passes Attempted         | 44.58    |           71 |
|  9 | Pass Completion %        | 78.6%    |           72 |
| 10 | Progressive Passes       | 3.38     |           36 |
| 11 | Progressive Carries      | 4.97     |           80 |
| 12 | Successful Take-Ons      | 2.83     |           87 |
| 13 | Touches (Att Pen

In [96]:
API_KEY=userdata.get('API_KEY')

In [97]:
"""
At the command line, only need to run once to install the package via pip:

$ pip install google-generativeai
"""

import google.generativeai as genai
genai.configure(api_key=API_KEY)

# Set up the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]



In [98]:
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

convo = model.start_chat(history=[
])

In [99]:
convo.send_message(prompt)
# print(convo.last.text)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=glm.GenerateContentResponse({'candidates': [{'content': {'parts': [{'text': "## Scouting Report: Lamine Yamal\n\n### Player Information\n\n* **Name:** Lamine Yamal\n* **Position:** Winger\n* **Age:** 16\n* **Team:** Barcelona\n\n### Strengths and Weaknesses\n\n**Technical:**\n\n* **Strengths:** Exceptional dribbling skills, close control, and ability to beat defenders in 1v1 situations. Accurate passing and vision to create scoring opportunities. Possesses a strong shot with both feet. \n* **Weaknesses:** Decision-making in the final third can be inconsistent, sometimes opting for low-percentage shots or passes. Needs to refine his crossing technique for better accuracy.\n\n**Tactical:**\n\n* **Strengths:** Intelligent movement off the ball, finding space between defenders and exploiting channels. Demonstrates good understanding of positional play and rotations. \n* **Weaknesses:** Can be caught ball-watchi

In [100]:
from pathlib import Path
from IPython.display import Markdown


In [101]:
display(Markdown(convo.last.text))

## Scouting Report: Lamine Yamal

### Player Information

* **Name:** Lamine Yamal
* **Position:** Winger
* **Age:** 16
* **Team:** Barcelona

### Strengths and Weaknesses

**Technical:**

* **Strengths:** Exceptional dribbling skills, close control, and ability to beat defenders in 1v1 situations. Accurate passing and vision to create scoring opportunities. Possesses a strong shot with both feet. 
* **Weaknesses:** Decision-making in the final third can be inconsistent, sometimes opting for low-percentage shots or passes. Needs to refine his crossing technique for better accuracy.

**Tactical:**

* **Strengths:** Intelligent movement off the ball, finding space between defenders and exploiting channels. Demonstrates good understanding of positional play and rotations. 
* **Weaknesses:** Can be caught ball-watching defensively and needs to improve his awareness of surrounding players. Decision-making regarding when to dribble, pass, or shoot requires further development.

**Physical:**

* **Strengths:**  Excellent agility, balance, and quick acceleration. Shows good stamina and work rate throughout the match.
* **Weaknesses:** Still developing physically, lacks the strength and power of more mature players. 

**Mental:**

* **Strengths:**  Displays confidence and composure on the field. Possesses a strong work ethic and willingness to learn. 
* **Weaknesses:**  His temperament in challenging situations requires further observation.

### Potential

Lamine Yamal possesses immense potential to become a top-class winger. His technical skills, combined with his tactical understanding and physical attributes, provide a strong foundation for development. With continued refinement of his decision-making, defensive awareness, and physical growth, he has the potential to be a star player.

**Areas for Improvement:**

* Decision-making in the final third.
* Defensive awareness and positioning.
* Crossing accuracy.
* Physical strength and power.

### Injury History

There is no readily available information about significant injuries or recurring issues for Lamine Yamal at this time. 

### Statistical Analysis

**Key Performance Indicators:**

* **Dribbling:** Yamal's successful take-ons per 90 (2.83) place him in the 87th percentile, highlighting his exceptional dribbling ability. 
* **Chance Creation:** His shot-creating actions (4.49 per 90) and xAG (0.28 per 90) demonstrate his effectiveness in generating scoring opportunities.
* **Passing:** While his pass completion percentage is respectable (78.6%), his progressive passes per 90 (3.38) are relatively low, suggesting room for improvement in forward passing and breaking defensive lines. 

**Contextualizing Statistics:**

It's important to consider Yamal's age and limited playing time at the senior level when analyzing his statistics. His numbers are impressive for a young player in a top-tier league and suggest a high ceiling for his potential.

### Comparison

Yamal's playing style and skillset draw comparisons to other young, dynamic wingers like Ansu Fati (Barcelona) and Rodrygo (Real Madrid). He shares their ability to take on defenders, create chances, and score goals.

### Summary

Lamine Yamal is a highly talented young winger with exceptional technical skills, tactical intelligence, and physical attributes. While he still has areas to develop, his potential is undeniable. He has the potential to become a key player for Barcelona and a star in world football.

### Recommendation

**Lamine Yamal is definitely a player worth pursuing further.**  Continued scouting and monitoring of his development are strongly recommended.  Depending on the team's needs and resources, exploring options for signing or loaning him in the future could be beneficial. 


In [102]:
# Print to file.
file_path = Path(f"{player_name}_scouting_report.md")
file_path.write_text(convo.last.text)

3765

# Future Plans
### Include Football Manager stats from internet
* https://fminside.net/players/4-fm-24/2000256231-lamine-yamal

### Include video understanding with Gemini 1.5 pro
* https://github.com/google-gemini/cookbook/blob/main/quickstarts/Video.ipynb

### Search internet?
* Search grounding?
